<img src="logo.png" style="width: 50%">

<font color='blue', size=5.5>Ipython Notebook para mostrar como ejecutar el protocolo de tratamiento de imágenes Landsat del Laboratorio de SIG y Teledetección de la Estación Biológica de Doñana </font>


#### Todo el protocolo está incluido dentro de una clase (Protocolo) , la cual incluye como métodos (funciones de la clase) los distintos pasos que se han seguido tradicionalmente (Importación, Reproyección, Corrección Radiometrica y Normalización).

#### Para ejecutar el protocolo se usan distintas librerías de python (fundamentalmente <font color='red'>GDAL</font> y <font color='red'>Rasterio</font>) y el software <font color='red'>Miramon</font> (importación y Corrección Radiométrica). Esté último software se llama mediante la generación y ejecución de archivos de texto en formato .bat

#### Lo primero que debemos hacer para poder ejecutar el protocolo es importarlo, para ello lo único que tenemos que hacer es llamar a la clase incluida en el archivo python. Esto lo hacemos del siguiente modo:


In [1]:
from Protocolo_V25 import Protocolo

#### Ahora ya tenemos disponible la clase y todos sus métodos. En principio para ejecutarla debemos instanciarla indicándole la ruta en la que está la imagen (o imágenes) a procesar. Un criterio a prori necesario es que se cree la estructura de carpetas /ori, /geo, /rad, /nor debiendo estar nuestra imagen a procesar en la carpeta ori. Si las carpetas no estuvieran creadas se pueden crear automáticamente, pero si es necesario que la imagen esté en una carpeta llamada ori


#### Antes de instanciar la clase hay que comprobar que el servidor de la Base de Datos esá activado,  activarlo si no lo estuviera. Esto lo hacemos abriendo un cmd de windows y escribiendo <font color='red'>"mongod"</font>

In [2]:
myProtocolo = Protocolo(r'C:\Users\Diego\Desktop\Presentacion\protocolo\ori\20140812l8oli202_34')

El porcentaje de nubes en la escena es de 0.1


#### Con esta línea ya tenemos nuestro protocolo iniciado. Con esto ya tenemos diponibles todos los métodos a los que podemos llamar con nuestra clase. Los principales obviamente son Importación, Reproyección, Corrad y Normalización, pero estos se componen de varios métodos, a los que si quisieramos podríamos llamar de forma independiente. Para ver los métodos disponibles desde un Notebook tenemos que poner el nombre que le hayamos dado a la instancia (myProtocolo en este caso) seguido de un punto y darle al tabulador.  
#### Otro aspecto importante que se crea al instanciar la clase es el registro de la escena en la Base de Datos . En concreto se almacena el id, la fecha en la que se instancia la clase y la cantidad de nubes que hay tanto en la escena (sacado del MTL.txt provisto con la escena) como sobre Doñana (Parque Nacional). Este valor lo sacamos nosotros cruzando la máscara de nubes con el shapefile del Parque Nacional

#### Es importante señalar que si se quisiera ejecutar el protocolo sobre todas las escenas en un directorio sería fácilmente ejecutable del siguiente modo (en principio esto estará implementado en la opción de visualización/ejecución que se decida):

In [ ]:
#Con estas 3 líneas se ejecutaría todo el protocolo a todas las escenas en un directorio

for i in os.listdir(path/to/folder/containig/severals/scenes):
    
    myProtocolo = os.path.join(path, i)
    myProtocolo.run_all()

#### El primer paso que haremos será la importación. Este paso se divide en varios, lo primero que hará será generar la máscara de nubes y calcular el porcentaje de cobertura sobre el Parque Nacional de Doñana. Luego hará la importación de los Getiff originales a través de un bat de Miramon. En la importación también calcula el kl (valor mínimo) dentro de unos polígonos definidos (embalses y sombras de sierras), escribiendo un archivo de texto kl.rad, que será usado en la Corección Radiométrica

In [3]:
myProtocolo.importacion()

comenzando Fmask
Mascara de nubes generada en -60.5929999352 segundos
El porcentaje de nubes en el Parque Nacional es de 0.0
['C:\\MiraMon\\TIFIMG', '9', 'C:\\Users\\Diego\\Desktop\\Presentacion\\protocolo\\ori\\20140812l8oli202_34\\LC82020342014224LGN00_B1.TIF', 'C:\\Users\\Diego\\Desktop\\Presentacion\\protocolo\\ori\\20140812l8oli202_34\\miramon_import', '1', '0', '/MD=C:\\Users\\Diego\\Desktop\\Presentacion\\protocolo\\ori\\20140812l8oli202_34\\LC82020342014224LGN00_MTL.txt', '/DT=c:\\MiraMon']
Escena importada con exito en 26.2680001259 segundos
modificados los metadatos del rad
Mascara de nubes, importacion y obtencion de kls concluido en  114.860000134 segundos


#### En segundo lugar realizaremos la reproyección, este es el paso que más tiempo conlleva, más o menos 1 minuto por cada banda. Junto con la reproyección copia y modifica los .dco y el rel, para que tenga los valores adecuados. La máscara de nubes la reproyecta y la guarda directamente en la carpeta de la escena en nor

In [4]:
myProtocolo.reproyeccion()

Reproyectando LC82020342014224LGN00_B1.TIF
LC82020342014224LGN00_B1.TIF reproyectada en 54.2590000629 segundos
Reproyectando LC82020342014224LGN00_B10.TIF
LC82020342014224LGN00_B10.TIF reproyectada en 57.5209999084 segundos
Reproyectando LC82020342014224LGN00_B11.TIF
LC82020342014224LGN00_B11.TIF reproyectada en 55.3500001431 segundos
Reproyectando LC82020342014224LGN00_B2.TIF
LC82020342014224LGN00_B2.TIF reproyectada en 57.8669998646 segundos
Reproyectando LC82020342014224LGN00_B3.TIF
LC82020342014224LGN00_B3.TIF reproyectada en 53.9029998779 segundos
Reproyectando LC82020342014224LGN00_B4.TIF
LC82020342014224LGN00_B4.TIF reproyectada en 52.9479999542 segundos
Reproyectando LC82020342014224LGN00_B5.TIF
LC82020342014224LGN00_B5.TIF reproyectada en 57.2849998474 segundos
Reproyectando LC82020342014224LGN00_B6.TIF
LC82020342014224LGN00_B6.TIF reproyectada en 58.9549999237 segundos
Reproyectando LC82020342014224LGN00_B7.TIF
LC82020342014224LGN00_B7.TIF reproyectada en 53.2929999828 segund

#### En el siguiente realizamos la Corrección Radiométrica con Miramon. La salida en Float-32 la pasamos a byte para poder realizar la normalización en el útlimo paso

In [5]:
myProtocolo.Corrad()

Archivos copiados y renombrados a Rad
['C:\\MiraMon\\CORRAD', '1', 'C:\\Users\\Diego\\Desktop\\Presentacion\\protocolo\\rad\\20140812l8oli202_34\\20140812l8oli202_34_g_b1.img', 'C:\\Users\\Diego\\Desktop\\Presentacion\\protocolo\\rad\\20140812l8oli202_34', 'C:\\Users\\Diego\\Desktop\\Presentacion\\protocolo\\rad\\sindato.img', 'C:\\Users\\Diego\\Desktop\\Presentacion\\protocolo\\rad\\kl.rad', '/MULTIBANDA /CONSERVAR_MDT /LIMIT_LAMBERT=73.000000 /REF_SUP_100 /REF_INF_0 /DT=c:\\MiraMon']
Llamando a Miramon... Miramon!!!!!!
Escena corregida con exito en 85.5249998569 segundos
modificados los metadatos de  20140812l8oli202_34_gr_b1.hdr
modificados los metadatos de  20140812l8oli202_34_gr_b2.hdr
modificados los metadatos de  20140812l8oli202_34_gr_b3.hdr
modificados los metadatos de  20140812l8oli202_34_gr_b4.hdr
modificados los metadatos de  20140812l8oli202_34_gr_b5.hdr
modificados los metadatos de  20140812l8oli202_34_gr_b6.hdr
modificados los metadatos de  20140812l8oli202_34_gr_b7.hdr


C:\Anaconda\lib\site-packages\rasterio\__init__.py:103: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


#### Finalmenterealizamos la normalización. Al final de este paso la máscara de nubes se guardará también como máscara binaria de 0 y 1.

In [3]:
myProtocolo.normalizacion()

landsat 8

C:\Users\Diego\Desktop\Presentacion\protocolo\rad\20140812l8oli202_34\20140812l8oli202_34_gr_b2.img  desde normalize
comenzando nor1
C:\Users\Diego\Desktop\Presentacion\protocolo\nor\20140812l8oli202_34\20140812l8oli202_34_CM.img
mascara:  C:\Users\Diego\Desktop\Landsat_8\MASK_1.img
b2

++++++++++++++++++++++++++++++++++
slope: 1.89634391348 intercept: 10.4025564279 r 0.979764226155 N: 53152
++++++++++++++++++++++++++++++++++

{'Arena': 669, 'Embalses': 1581, 'Mineria': 87, 'Mar': 47721, 'Urbano-1': 476, 'Urbano-2': 735, 'Aeropuertos': 212, 'Pastizales': 82, 'Pinar': 1589} b2
parametros en nor1:  {'b2': {'Parametros': {'slope': 1.8963439134807982, 'r': 0.97976422615490433, 'intercept': 10.402556427860034, 'N': 53152}, 'Tipo_Area': {'Arena': 669, 'Embalses': 1581, 'Mineria': 87, 'Mar': 47721, 'Urbano-1': 476, 'Urbano-2': 735, 'Aeropuertos': 212, 'Pastizales': 82, 'Pinar': 1589}}}
\comenzando nor2


Normalizacion de  b2  realizada.

C:\Users\Diego\Desktop\Presentacion\protocol

C:\Anaconda\lib\site-packages\rasterio\__init__.py:103: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)
